In [5]:
import pickle as pkl
import torch
import os

In [23]:
def get_data(prefix, base_path='./'):
    in_dir = os.listdir(base_path)
    in_dir = [f for f in in_dir if f.startswith(prefix)]
    in_dir = sorted(in_dir)
    data = []
    for f in in_dir:
        with open(base_path + f, 'rb') as file:
            data.append(pkl.load(file))
    return data

In [24]:
tedlium_test = get_data('tedlium_test')

In [28]:
wer_tedlium_test = [d['wer'] for d in tedlium_test]
sum(wer_tedlium_test)/3

0.057762813318368876

In [25]:
tedlium_test[0].keys()

dict_keys(['wer', 'words', 'ins_rate', 'del_rate', 'sub_rate', 'model_output', 'gold', 'args_dict', 'repeat'])

In [22]:
tedlium_test[0]['ins_rate']

0.008683324472798157